In [1]:
import sys,os
import concurrent.futures
import getpass
from selenium import webdriver #install selenium
from selenium.webdriver.common.keys import Keys
import time,random,string
#put geckodriver into the working directory
geckodriver_path=(os.path.join(os.getcwd(),'geckodriver-dir'))#this is wrong search path for modules
geckodriver_path=":"+geckodriver_path
os.environ["PATH"]+=geckodriver_path





In [118]:
#put geckodriver into the working directory
chromedriver_path=(os.path.join(os.getcwd(),'chrome'))#this is wrong search path for modules
chromedriver_path=":"+chromedriver_path
os.environ["PATH"]+=chromedriver_path

In [45]:
def current_status(driver):
    result={}
    for col in range(1,5):
        for row in range(1,5):
            key="c%d_r%d" %(col,row)
            try:
                tile=driver.find_element_by_class_name("tile-position-%d-%d" %(col,row))
                value=tile.find_element_by_class_name("tile-inner").text
                result[key]=int(value)
            except:
                result[key]=0
    return result
def get_score(driver):
    try:
        value=driver.find_element_by_class_name("score-container").text
        result=int(value)
    except:
        result=0
    return result

def game_over(driver):
    element=driver.find_element_by_class_name('game-message')
    display=element.value_of_css_property('display')
    return display=='none'

def game_retry(driver):
    retry_button=driver.find_element_by_class_name("retry-button")
    retry_button.click()
    
    


In [154]:
import copy
keymap={'up':Keys.ARROW_UP,
        'down':Keys.ARROW_DOWN,
        'left':Keys.ARROW_LEFT,
        'right':Keys.ARROW_RIGHT}
driver = webdriver.PhantomJS()
driver.get("https://wayneyeye.github.io/2048/")


driver_s = webdriver.PhantomJS()
driver_s.get("https://wayneyeye.github.io/2048_shadow/")






In [178]:
try_score={}
def shadow_and_try(driver_s,gameState,try_score,depth=1):
    try_score['Max']=0
    if not game_over(driver_s):
        score=get_score(driver_s)
        game_retry(driver_s)
        return score
    if depth==0: 
        score=get_score(driver_s)
        return score
    for move in ['up','down','left','right']:
#         print(move)
        try_score[move]={}
        js_stmt="new GameManager(4, KeyboardInputManager, HTMLActuator, LocalStorageManager,%s);" %gameState
        driver_s.execute_script(js_stmt)
#         print("reset")
#         time.sleep(5)
        driver_s.find_element_by_class_name('container').send_keys(keymap[move])
#         time.sleep(5)
        gameState2=driver_s.execute_script("return localStorage.gameState;")
        if gameState==gameState2:
#             print(gameState)
#             print(gameState2)
            continue
        try_score['Max']=max(shadow_and_try(driver_s,gameState2,try_score[move],depth=depth-1),try_score['Max'])
    return try_score['Max']

In [155]:
gameState=driver.execute_script("return localStorage.gameState;")

In [156]:
import pprint
try_score={}
gameState=driver.execute_script("return localStorage.gameState;")
shadow_and_try(driver_s,gameState,try_score,depth=3)
pprint.pprint(try_score)

{'Max': 64,
 'down': {'Max': 36,
          'down': {'Max': 0,
                   'down': {'Max': 0},
                   'left': {'Max': 0},
                   'right': {'Max': 0},
                   'up': {'Max': 0}},
          'left': {'Max': 36,
                   'down': {'Max': 0},
                   'left': {'Max': 0},
                   'right': {'Max': 0},
                   'up': {'Max': 0}},
          'right': {'Max': 0,
                    'down': {'Max': 0},
                    'left': {'Max': 0},
                    'right': {'Max': 0},
                    'up': {'Max': 0}},
          'up': {'Max': 0,
                 'down': {'Max': 0},
                 'left': {'Max': 0},
                 'right': {'Max': 0},
                 'up': {'Max': 0}}},
 'left': {'Max': 56,
          'down': {'Max': 52,
                   'down': {'Max': 0},
                   'left': {'Max': 0},
                   'right': {'Max': 0},
                   'up': {'Max': 0}},
          'left': {'Max

In [108]:
driver.close()
driver_s.close()

In [165]:
def predit_next_move(try_score):
    ranklist=[]
    for move in ['up','down','left','right']:
        ranklist.append((move,try_score[move]['Max']))
    return sorted(ranklist, key=lambda movement: movement[1],reverse=True)

In [166]:
predit_next_move(try_score)

[('up', 20), ('left', 12), ('down', 4), ('right', 0)]

In [180]:
import copy
keymap={'up':Keys.ARROW_UP,
        'down':Keys.ARROW_DOWN,
        'left':Keys.ARROW_LEFT,
        'right':Keys.ARROW_RIGHT}
driver = webdriver.Firefox()
driver.get("https://wayneyeye.github.io/2048/")


while game_over(driver):
    driver_s = webdriver.PhantomJS()
    driver_s.get("https://wayneyeye.github.io/2048_shadow/")
    try_score={}
    gameState=driver.execute_script("return localStorage.gameState;")
    shadow_and_try(driver_s,gameState,try_score,depth=4)
    predict_move=predit_next_move(try_score)
    print(predict_move)
    driver_s.close()
    for j in range(4):
        print(predict_move[j][0])
        driver.find_element_by_class_name('container').send_keys(keymap[predict_move[j][0]])
        gameState2=driver.execute_script("return localStorage.gameState;")
        if gameState!=gameState2:
            break
        else:
            print("no improvement!")
    


[('left', 20), ('down', 12), ('up', 8), ('right', 8)]
left
[('right', 16), ('up', 8), ('down', 8), ('left', 0)]
right
[('up', 20), ('down', 20), ('right', 20), ('left', 16)]
up
[('up', 24), ('left', 24), ('right', 20), ('down', 16)]
up
[('up', 24), ('left', 24), ('right', 24), ('down', 20)]
up
[('left', 36), ('right', 28), ('up', 24), ('down', 24)]
left
[('up', 36), ('down', 32), ('left', 32), ('right', 24)]
up
[('left', 52), ('down', 36), ('right', 36), ('up', 8)]
left
[('down', 52), ('up', 40), ('right', 36), ('left', 28)]
down
[('up', 56), ('left', 56), ('right', 56), ('down', 40)]
up
[('left', 64), ('up', 48), ('down', 40), ('right', 40)]
left
[('right', 72), ('left', 68), ('up', 64), ('down', 56)]
right
[('right', 76), ('up', 72), ('left', 72), ('down', 68)]
right
[('up', 124), ('down', 120), ('right', 96), ('left', 76)]
up
[('left', 140), ('up', 136), ('down', 128), ('right', 104)]
left
[('left', 152), ('up', 136), ('down', 128), ('right', 128)]
left
[('up', 128), ('down', 128), 

WebDriverException: Message: {"errorMessage":"Can't find variable: None","request":{"headers":{"Accept":"application/json","Accept-Encoding":"identity","Connection":"close","Content-Length":"160","Content-Type":"application/json;charset=UTF-8","Host":"127.0.0.1:34415","User-Agent":"Python http auth"},"httpVersion":"1.1","method":"POST","post":"{\"script\": \"new GameManager(4, KeyboardInputManager, HTMLActuator, LocalStorageManager,None);\", \"args\": [], \"sessionId\": \"c18aae90-6b70-11e7-b255-111e39591748\"}","url":"/execute","urlParsed":{"anchor":"","query":"","file":"execute","directory":"/","path":"/execute","relative":"/execute","port":"","host":"","password":"","user":"","userInfo":"","authority":"","protocol":"","source":"/execute","queryKey":{},"chunks":["execute"]},"urlOriginal":"/session/c18aae90-6b70-11e7-b255-111e39591748/execute"}}
Screenshot: available via screen
